# Medicare Part D - Prescription ETL
---
- placeholder text for description
---
### Dependencies and Setup

In [ ]:
# TC


In [ ]:
# EH


In [ ]:
# TMI


In [ ]:
# GO


---
## Extraction

In [ ]:
# CMS Part D Prescriber Summary Reports


In [ ]:
# Medicaid.gov - National Average Drug Acquisition Cost


In [ ]:
# Q1Medicare
q1_url = 'https://q1medicare.com/PartD-The-2019-Medicare-Part-D-Outlook.php#ME963C921251342D6BE66AB6E656C89F6'


---
## Transformation

In [ ]:
# CMS Clean-up:


In [ ]:
# Medicaid Clean-up:


In [ ]:
# Q1 Clean-up:


---
## Loading